In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import math

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

def count_fingers(hand_landmarks):
    tips = [4, 8, 12, 16, 20]
    fingers = []

    if hand_landmarks.landmark[tips[0]].x < hand_landmarks.landmark[tips[0] - 1].x:
        fingers.append(1)
    else:
        fingers.append(0)

    for tip in tips[1:]:
        if hand_landmarks.landmark[tip].y < hand_landmarks.landmark[tip - 2].y:
            fingers.append(1)
        else:
            fingers.append(0)

    return sum(fingers)

def distance_between_fingers(hand_landmarks, i1, i2):
    x1 = hand_landmarks.landmark[i1].x
    y1 = hand_landmarks.landmark[i1].y
    x2 = hand_landmarks.landmark[i2].x
    y2 = hand_landmarks.landmark[i2].y
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

bg_color = (0, 0, 0)
circle_pos = [320, 240]

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    output = np.full_like(frame, bg_color, dtype=np.uint8)
    h, w, _ = frame.shape

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            fingers_up = count_fingers(hand_landmarks)
            dist_thumb_index = distance_between_fingers(hand_landmarks, 4, 8)

            if fingers_up == 5:
                bg_color = (0, 128, 255)

            elif fingers_up == 0:
                bg_color = (0, 0, 0)

            elif fingers_up == 2 and dist_thumb_index < 0.08:
                circle_pos[0] += 10

            elif fingers_up == 1:
                circle_pos[0] -= 10

    overlay = output.copy()
    cv2.circle(overlay, tuple(circle_pos), 40, (255, 255, 255), -1)
    blended = cv2.addWeighted(overlay, 1, frame, 0.5, 0)

    cv2.imshow("Hand Gesture Control", blended)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
